In [1]:
import glob
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
import datetime
import dateutil.parser as parser
%matplotlib inline
plt.style.use('ggplot')

/Users/amandazhang/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [7]:
master = pd.read_csv('master.csv', index_col=0)

In [21]:
master['log_commute_time'] = -np.log(master.commute_time)

In [23]:
master = master.drop('commute_time', axis=1)

In [24]:
master.head()

,zika_counts,temp,pop,mosquito,total_seats_oag,flights_oag,log_commute_time
location,,,,,,,
Alabama,165,78.6,4863300,55.0,0.0,0.0,-7.374302
Alaska,0,52.3,741894,0.0,0.0,0.0,-7.530105
American Samoa,4,86.0,54343,12.0,0.0,0.0,-11.772244
Arizona,175,78.1,6931071,48.0,0.0,0.0,-6.358312
Arkansas,117,78.8,2988248,25.0,0.0,0.0,-7.620608


In [37]:
nsims = 2
R2s_base_train = []
R2s_base_test = []
R2s_flightsonly_train = []
R2s_flightsonly_test = []
R2s_commuteonly_train = []
R2s_commuteonly_test = []

for i in range(nsims):
    # make x and y
    y = master.zika_counts
    x = master.drop('zika_counts', axis=1)
    x_flightsonly = x[['temp','pop','mosquito','flights_oag']]
    x_commuteonly = x[['temp','pop','mosquito','log_commute_time']]
    #x_flightsonly = x[['flights_oag']]
    #x_commuteonly = x[['log_commute_time']]

    # scale
    x = preprocessing.scale(x)
    x_flightsonly = preprocessing.scale(x_flightsonly)
    x_commuteonly = preprocessing.scale(x_commuteonly)

    # split into train and test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    x_flightsonly_train, x_flightsonly_test, y_flightsonly_train, y_flightsonly_test = train_test_split(x_flightsonly, y, test_size=0.1)
    x_commuteonly_train, x_commuteonly_test, y_commuteonly_train, y_commuteonly_test = train_test_split(x_commuteonly, y, test_size=0.1)
    
    # baseline Regression
    model = linear_model.LassoCV(cv=30)
    model.fit(x_train,y_train)
    R2s_base_train.append(model.score(x_train,y_train))
    R2s_base_test.append(model.score(x_test,y_test))
    
    # Regression model flights only
    model_flightsonly = linear_model.LassoCV(cv=30)
    model_flightsonly.fit(x_flightsonly_train,y_flightsonly_train)
    R2s_flightsonly_train.append(model_flightsonly.score(x_flightsonly_train,y_flightsonly_train))
    R2s_flightsonly_test.append(model_flightsonly.score(x_flightsonly_test,y_flightsonly_test))
    
    # model commute time only
    model_commuteonly = linear_model.LassoCV(cv=30)
    model_commuteonly.fit(x_commuteonly_train,y_commuteonly_train)
    R2s_commuteonly_train.append(model_commuteonly.score(x_commuteonly_train,y_commuteonly_train))
    R2s_commuteonly_test.append(model_commuteonly.score(x_commuteonly_test,y_commuteonly_test))

print 'Regression Baseline model\n'
print 'R2 on training: ', np.mean(R2s_base_train)
print 'R2 on testing: ', np.mean(R2s_base_test)
print '\nRegression Flights-only model\n'
print 'R2 on training: ', np.mean(R2s_flightsonly_train)
print 'R2 on testing: ', np.mean(R2s_flightsonly_test)
print '\nRegression Commute-time-only model\n'
print 'R2 on training: ', np.mean(R2s_commuteonly_train)
print 'R2 on testing: ', np.mean(R2s_commuteonly_test)

Regression Baseline model

R2 on training:  0.847691387055
R2 on testing:  0.638446379736

Regression Flights-only model

R2 on training:  0.834662881864
R2 on testing:  -0.00122591645345

Regression Commute-time-only model

R2 on training:  0.344727106507
R2 on testing:  0.295685400119


In [35]:
x.shape

(54, 6)